In [1]:
import pyomo.environ as pe
import os 

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"

In [2]:
m = pe.ConcreteModel()
m.x = pe.Var(within=pe.NonNegativeReals)
m.y = pe.Var(bounds=(0, None))
m.obj = pe.Objective(expr=4 * m.x + 3 * m.y, sense=pe.maximize)
m.c1 = pe.Constraint(expr=m.x <= 4)
m.c2 = pe.Constraint(expr=2 * m.x + m.y <= 10)
m.c3 = pe.Constraint(expr=(None, m.x + m.y, 10))
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT, datatype=pe.Suffix.INT)

pe.SolverFactory("gurobi", solver_io="lp").solve(m)

{'Problem': [{'Name': 'x1', 'Lower bound': 30.0, 'Upper bound': 30.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 2, 'Number of nonzeros': 5, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0019669532775878906, 'Error rc': 0}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [5]:
m.dual.display()

dual : Direction=IMPORT, Datatype=INT
    Key : Value
     c1 :   0.0
     c2 :   3.0
     c3 :   0.0


In [6]:
print(m.dual[m.x])

KeyError: 'x (key=139689920821104)'

In [ ]:
import pyomo.environ as aml

model = aml.ConcreteModel()
model.x = aml.Var(bounds=(0,1))
model.o = aml.Objective(expr=model.x + 27, sense=aml.maximize)
model.c = aml.Constraint(expr=model.x >= -1)

model.rc = aml.Suffix(direction=aml.Suffix.IMPORT)

gurobi = aml.SolverFactory("gurobi")
results = gurobi.solve(model)
assert str(results.solver.termination_condition) == "optimal"

print(model.rc[model.x])

1.0


In [ ]:
model.rc.display()

rc : Direction=IMPORT, Datatype=FLOAT
    Key : Value
      x :   1.0


In [11]:
from pyomo.environ import *
model = ConcreteModel()

# for access to dual solution for constraints
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
_ = SolverFactory('gurobi').solve(model, tee = False).write()

str = "   {0:7.2f} {1:7.2f} {2:7.2f} {3:7.2f}"

# print("Constraint  value  lslack  uslack    dual")
# for c in [model.demand, model.laborA, model.laborB]:
#     print(c, str.format(c(), c.lslack(), c.uslack(), model.dual[c]))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject 

In [15]:

import pandas as pd

import pyomo.environ as pyo
import pyomo.gdp as gdp #necessary if you use booleans to select active and inactive units

def bounds_rule(m, n, param='Cap_MW'):
    # m because it pases the module
    # n because it needs a variable from each set, in this case there was only m.N
    return (0, Gen[n][param]) #returns lower and upper bounds.

def unit_commitment():
    m = pyo.ConcreteModel()
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)
    N=Gen.keys()
    m.N = pyo.Set(initialize=N)    
    
    m.Pgen = pyo.Var(m.N, bounds = bounds_rule) #amount of generation
    m.Rgen = pyo.Var(m.N, bounds = bounds_rule) #amount of generation

    # m.OnOff = pyo.Var(m.N, domain=pyo.Binary) #boolean on/off marker
    
    # objective
    m.cost = pyo.Objective(expr = sum( m.Pgen[n]*Gen[n]['energy_$MWh'] + m.Rgen[n]*Gen[n]['res_$MW'] for n in m.N), sense=pyo.minimize)
    
    # demand
    m.demandP = pyo.Constraint(rule=lambda m: sum(m.Pgen[n] for n in N) == Demand['ener_MWh'])
    m.demandR = pyo.Constraint(rule=lambda m: sum(m.Rgen[n] for n in N) == Demand['res_MW'])
    
    # machine production limits
    # m.lb = pyo.Constraint(m.N, rule=lambda m, n: Gen[n]['Cap_min']*m.OnOff[n] <= m.Pgen[n]+m.Rgen[n] )
    # m.ub = pyo.Constraint(m.N, rule=lambda m, n: Gen[n]['Cap_MW']*m.OnOff[n] >= m.Pgen[n]+m.Rgen[n])
    m.lb = pyo.Constraint(m.N, rule=lambda m, n: Gen[n]['Cap_min'] <= m.Pgen[n]+m.Rgen[n] )
    m.ub = pyo.Constraint(m.N, rule=lambda m, n: Gen[n]['Cap_MW'] >= m.Pgen[n]+m.Rgen[n])

    m.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    return m

Gen = {
    'GenA' : {'Cap_MW': 100, 'energy_$MWh': 10, 'res_$MW': 0, 'Cap_min': 0},
    'GenB' : {'Cap_MW': 100, 'energy_$MWh': 30, 'res_$MW': 25, 'Cap_min': 0},
} #starting data

Demand = {
    'ener_MWh': 130, 'res_MW': 20,
} #starting data
m = unit_commitment()
_ = pyo.SolverFactory('gurobi').solve(m).write() 
# m.display()


df = pd.DataFrame.from_dict([m.Pgen.extract_values(), m.Rgen.extract_values()]).T.rename(columns={0: "P", 1: "R"})
print(df)
print("Cost Function result: " + str(m.cost.expr()) + "$.")

print(m.rc.display())
print(m.dual.display())
print(m.dual[m.demandR])

da= {'duals': m.dual[m.demandP],
     'uslack': m.demandP.uslack(),
     'lslack': m.demandP.lslack(),
    }
db= {'duals': m.dual[m.demandR],
     'uslack': m.demandR.uslack(),
     'lslack': m.demandR.lslack(),
    }

duals = pd.DataFrame.from_dict([da, db]).T.rename(columns={0: "demandP", 1: "demandR"})

print(duals)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 2300.0
  Upper bound: 2300.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 4
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 4
  Number of nonzeros: 12
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject

TypeError: 'str' object is not callable

In [16]:
print(m.rc.display())
print(m.dual.display())

rc : Direction=IMPORT, Datatype=FLOAT
    Key        : Value
    Pgen[GenA] :   0.0
    Pgen[GenB] :   0.0
    Rgen[GenA] :   0.0
    Rgen[GenB] :   5.0
None
dual : Direction=IMPORT_EXPORT, Datatype=FLOAT
    Key      : Value
     demandP :  30.0
     demandR :  20.0
    lb[GenA] :   0.0
    lb[GenB] :   0.0
    ub[GenA] : -20.0
    ub[GenB] :   0.0
None
